In [ ]:
import numpy as np
import re
import pickle
import gzip

import sys
sys.path.append('/Users/chenwei/Desktop/Github/ViDa') 

import imp, vida.data_processing.utils
imp.reload(vida.data_processing.utils)
from vida.data_processing.utils import *

import imp, vida.adjmat.dp2adj
imp.reload(vida.adjmat.dp2adj)
from vida.adjmat.dp2adj import *

import imp, vida.data_processing.comp_time
imp.reload(vida.data_processing.comp_time)
from vida.data_processing.comp_time import *


### three way

In [ ]:
fpath = '/Users/chenwei/Desktop/Github/ViDa//data/raw_data/Machinek-data'
rxn = 'Machinek-PRF'
ref_strands = 'CCCTCCACATTCAACCTCAAACTCACC+TGGTGTTTGTGGGTGTGGTGAGTTTGAGGTTGA+GGTGAGTTTGAGGTTGAATGTGGA'
strand_a = 'CCCTCCACATTCAACCTCAAACTCACC'  # substrate_perf_seq
strand_b = 'TGGTGTTTGTGGGTGTGGTGAGTTTGAGGTTGA'  # incumbent_perf_seq
strand_c = 'GGTGAGTTTGAGGTTGAATGTGGA'  # invader_perf_seq
strand_list = [strand_a, strand_b, strand_c]

ref_name_list = assign_base_names(ref_strands)
ref_name = [item for sublist in ref_name_list for item in sublist]


In [ ]:
strand_list

In [ ]:
# f = open('/Users/chenwei/Desktop/Github/ViDa/data/raw_data/Machinek-data/Machinek-PRF/Machinek-PRF-0.txt', 'r')
# lines = f.read().splitlines()
# part_strand = 'GGTGAGTTTGAGGTTGAATGTGGA'
# seq_line = []
# pattern = re.compile(r'^(.*?)\s+t=([\d.e+-]+) seconds, dG=([-\d.e+]+) kcal/mol')

### read files

In [ ]:
trajs_seqs, trajs_states, trajs_times, trajs_energies = read_machinek(fpath,rxn,strand_a,num_files=1)
trajs_seqs.shape, trajs_states.shape

In [ ]:
trajs_seqs[0][-10:]

### preprocess

In [ ]:
dp_arr, dp_og, pair, energy, trans_time = concat_machinek(trajs_states, trajs_times, trajs_energies)
dp_uniq, dp_og_uniq, pair_uniq, energy_uniq, indices_uniq, indices_all = get_uniq(dp_arr, dp_og, pair, energy)
adj_uniq, seqlabel_uniq = sim_adj_3strand_uniq(dp_arr, trajs_seqs, ref_name, ref_name_list, strand_list, indices_uniq)    

#### adj_uqni debug

In [ ]:
dp_arr.shape, dp_arr[0].shape, trajs_seqs.shape, len(trajs_seqs[0]), adj_uniq.shape, seqlabel_uniq.shape, indices_all.shape

In [ ]:
la = 980
lb = la+10

seqlabel_uniq[la:lb], dp_og_uniq[la:lb]

In [ ]:
dp_og_uniq[-1], adj_uniq[-1]

In [ ]:
dp_og_uniq[-1], dp_uniq[-1], seqlabel_uniq[-1]

In [ ]:
"""
## TEST 1
## Calculate the adjacency matrix for the individual structure ##

"""
dp_structure = dp_uniq[-1]

alter_name_arr = concat_disorder(trajs_seqs[0],ref_name_list, strand_list)

alter_name_arr_test = alter_name_arr[-1]

alter_name = np.concatenate(alter_name_arr_test)

dplast = dp2adj_3strand(ref_name, alter_name, alter_name_arr_test, dp_structure)
dplast

In [ ]:
"""
## TEST 2
## Calculate the whole adjacency matrix than compared with uniq ones ##

"""
dppmat = sim_adj_3strand(dp_arr, trajs_sequences, ref_name, ref_name_list, strand_list)
dppmat_uniq = dppmat[indices_uniq]

np.all(adj_uniq == dppmat_uniq)

### UNCHANGE

### collect time

In [ ]:
hold_time, trj_id = sim_ht(trans_time)
hold_time_uniq = mean_holdingtime(hold_time, indices_uniq, indices_all)
cum_time_uniq,freq_uniq = cumu_holdingtime(hold_time, indices_uniq, indices_all)


### dp to adj

In [ ]:
adj_uniq = sim_adj_3strand_uniq(dp_arr, trajs_seqs, ref_name, ref_name_list, strand_list, indices_uniq)    


In [ ]:
combined_array = np.concatenate(adj, axis=0)
combined_array.shape

### Draw

In [ ]:
import networkx as nx

import matplotlib.pyplot as plt

# Create a graph from the adjacency matrix
G = nx.Graph(adj[10])

# Draw the graph
pos = nx.spring_layout(G)  # Positions for all nodes
nx.draw(G, pos=nx.kamada_kawai_layout(G), with_labels=True, font_weight='bold', node_size=200, node_color='green', font_size=10, font_color='black')
# nx.draw(G, with_labels=True, font_weight='bold', node_size=200, node_color='green', font_size=10, font_color='black')


# Show the plot
plt.show()


### Debug

In [9]:
import numpy as np
import re
import pickle
import gzip
import pandas as pd
import sys
sys.path.append('/Users/chenwei/Desktop/Github/ViDa') 


import imp, vida.data_processing.strandReorder
imp.reload(vida.data_processing.strandReorder)
from vida.data_processing.strandReorder import *

# import imp, vida.data_processing.utils
# imp.reload(vida.data_processing.utils)
# from vida.data_processing.utils import *

import imp, vida.adjmat.dp2adj
imp.reload(vida.adjmat.dp2adj)
from vida.adjmat.dp2adj import *

import imp, vida.data_processing.comp_time
imp.reload(vida.data_processing.comp_time)
from vida.data_processing.comp_time import *


In [2]:
datafile='Machinek-PRF-trunc'

ref_strands = 'CCCTCCACATTCAACCTCAAACTCACC+TGGTGTTTGTGGGTGTGGTGAGTTTGAGGTTGA+GGTGAGTTTGAGGTTGAATGTGGA'
strand_sub = 'CCCTCCACATTCAACCTCAAACTCACC'  # substrate_perf_seq
strand_incb = 'TGGTGTTTGTGGGTGTGGTGAGTTTGAGGTTGA'  # incumbent_perf_seq
strand_inv = 'GGTGAGTTTGAGGTTGAATGTGGA'  # invader_perf_seq

In [3]:
inpath = f'../data/post_data/{datafile}/preprocess_Machinek-PRF.npz'
loaded_data = np.load(inpath, allow_pickle=True)
dp_uniq = loaded_data['dp_uniq']
dp_og_uniq = loaded_data['dp_og_uniq']
shortname_uniq = loaded_data['shortname_uniq']
incbinvpair_uniq = loaded_data['incbinvpair_uniq']
indices_uniq = loaded_data['indices_uniq']
indices_all = loaded_data['indices_all']
trans_time = loaded_data["trans_time"]
energy_uniq = loaded_data["energy_uniq"]
ref_name = loaded_data["ref_name"]
ref_name_list = loaded_data["ref_name_list"]
strand_list = loaded_data["strand_list"]

inpath2 = f'../data/post_data/{datafile}/Machinek-PRF.pkl.gz'
with gzip.open(inpath2, 'rb') as file:
    load_data_seq = pickle.load(file)

ref_name = load_data_seq["ref_name"]
ref_name_list = load_data_seq["ref_name_list"]
strand_list = load_data_seq["strand_list"]
trajs_seqs = load_data_seq["trajs_seqs"]
trajs_states = load_data_seq['trajs_states']
trajs_times = load_data_seq['trajs_times']
trajs_energies = load_data_seq['trajs_energies']
trajs_shortnames = load_data_seq['trajs_shortnames']
trajs_incbinvpairs = load_data_seq['trajs_incbinvpairs']


In [13]:
df = pd.DataFrame(data={
                "Energy": energy_uniq, "DP": dp_og_uniq, 
                "IncbInvPair": incbinvpair_uniq, "ShortName": shortname_uniq,
                }
                )

In [21]:
color_mapping = {
        "inv+sub+incb": "grey",
        "incb+sub+inv": "red",
        "incb+sub inv": "blue",    
        "incb sub+inv": "orange",
    }
    

In [23]:
color_mapping[df['ShortName'][0]]

'grey'

In [7]:
unique_values, counts = np.unique(shortname_uniq, return_counts=True)
print(unique_values)  # Output: array of unique values
print(counts)  # Output: corresponding counts

['incb sub+inv' 'incb+sub inv' 'incb+sub+inv' 'inv+sub+incb']
[   32   611   178 15522]


In [20]:
len(trajs_seqs), len(trajs_shortnames), len(trajs_incbinvpairs), len(trajs_states), len(trajs_times), len(trajs_energies)

(400, 400, 400, 400, 400, 400)

In [44]:
dp_uniq.shape, dp_og_uniq.shape, shortname_uniq.shape, incbinvpair_uniq.shape, indices_uniq.shape, indices_all.shape, trans_time.shape

((16343,), (16343,), (16343,), (16343,), (16343,), (4363808,), (4363808,))

In [23]:
dp, dp_og, pair, energy, trans_time, seq, shortname, incbinvpair= concat_machinek(trajs_states, trajs_times, trajs_energies, trajs_seqs, trajs_shortnames, trajs_incbinvpairs)

dp.shape, pair.shape, dp_og.shape, energy.shape, trans_time.shape, seq.shape, shortname.shape, incbinvpair.shape,

((4363808,),
 (4363808,),
 (4363808,),
 (4363808,),
 (4363808,),
 (4363808,),
 (4363808,),
 (4363808,))

### dp2adj

In [89]:
inpath3 = f'../data/post_data/{datafile}/adjmat_Machinek-PRF.npz'
loaded_data3 = np.load(inpath3, allow_pickle=True)
adj_uniq = loaded_data3['adj_uniq']

adj_uniq.shape

(16343, 84, 84)

In [99]:
np.unique(adj_uniq.reshape(adj_uniq.shape[0], -1), axis=0).shape

(16343, 7056)

### Class

In [14]:
import re
from itertools import permutations
import numpy as np


class ThreeStrandReorder:
    
    @staticmethod
    def get_strand_pos(bpair):
        pattern = r'([a-z])(\d+)([a-z])(\d+)'
        match = re.match(pattern, bpair)

        if match:
            strand1, pos1, strand2, pos2 = match.groups()
            pos1 = int(pos1) - 1  # Convert to 0-based index
            pos2 = int(pos2) - 1  # Convert to 0-based index
            return strand1, pos1, strand2, pos2
        else:
            raise ValueError("Invalid base pair format")

    @staticmethod
    def hasIncbInvPair(base_pairs):
        has_bc = any('b' in pair and 'c' in pair for pair in base_pairs)

        return 1 if has_bc else 0
    
    
    def get_basepairs(self, seq, dp, ref_name_list, strand_list):
        def concat_disorder(seq, ref_name_list, strand_list):
            sequence_list = re.split(r'\s|\+', seq) 
            sequence = ''.join(sequence_list)

            for permuted_strand in permutations(strand_list):
                combined_sequence = ''.join(permuted_strand)
                if combined_sequence == sequence:
                    alter_name = [ref_name_list[strand_list.index(strand)] for strand in permuted_strand]
                    break

            return np.concatenate(alter_name)

        dp_structure = dp.replace(' ', '').replace('+', '')
        alter_name = concat_disorder(seq, ref_name_list, strand_list)

        stack = []
        base_pairs = []

        for name, char in zip(alter_name, dp_structure):
            if char == '(':
                stack.append(name)
            elif char == ')':
                if stack:
                    opening_index = stack.pop()
                    base_pairs.append(opening_index + name)
                else:
                    raise ValueError("Mismatched brackets")

        if stack:
            raise ValueError("Mismatched brackets")

        return base_pairs

    def reorderCase1(self, case1, dp, seq, short_seqname, ref_name_list, strand_list):
        """
        Reorder:
        'incb+inv+sub' -> 'inv+sub+incb' 
        'sub+incb+inv' -> 'inv+sub+incb'
        """
        
        position = case1.index(short_seqname)
        base_pairs = self.get_basepairs(seq, dp, ref_name_list, strand_list)
        
        incb_inv_pair = ThreeStrandReorder.hasIncbInvPair(base_pairs)
                
        if position == 0:   # 'inv+sub+incb': c+a+b ===> do nothing
            return dp, short_seqname, incb_inv_pair
        
        
        if position == 1:  # 'incb+inv+sub': b+c+a ===> c+a+b
            incb_list, inv_list, sub_list = [list(part) for part in re.split(r'\s|\+', dp)]
            
            for bpair in base_pairs:
                strand1, pos1, strand2, pos2 = ThreeStrandReorder.get_strand_pos(bpair)
                
                # only need to change incb (b) with its connected strands
                if strand1 == 'b' and strand2 == 'c':
                    incb_list[pos1] = ')'
                    inv_list[pos2] = '('

                if strand1 == 'b' and strand2 == 'a':
                    incb_list[pos1] = ')'
                    sub_list[pos2] = '('


        if position == 2: # 'sub+incb+inv': a+b+c ===> c+a+b
            sub_list, incb_list, inv_list = [list(part) for part in re.split(r'\s|\+', dp)]
            
            for bpair in base_pairs:
                strand1, pos1, strand2, pos2 = ThreeStrandReorder.get_strand_pos(bpair)
            
                # only need to change inv (c) with its connected strands
                if strand1 == 'a' and strand2 == 'c':
                    inv_list[pos2] = '('
                    sub_list[pos1] = ')'
                                    
                if strand1 == 'b' and strand2 == 'c':
                    inv_list[pos2] = '('
                    incb_list[pos1] = ')'
        
        inv = ''.join(inv_list)
        sub = ''.join(sub_list)
        incb = ''.join(incb_list)
        # inv+sub+incb
        dp_new = inv + "+" + sub + "+" + incb
        
        return dp_new, case1[0], incb_inv_pair


    def reorderCase2(self, case2, dp, seq, short_seqname, ref_name_list, strand_list):
        """
        Reorder:
        'inv+incb+sub' -> 'incb+sub+inv' 
        'sub+inv+incb' -> 'incb+sub+inv'
        """
        
        position = case2.index(short_seqname)
        base_pairs = self.get_basepairs(seq, dp, ref_name_list, strand_list)
        
        incb_inv_pair = ThreeStrandReorder.hasIncbInvPair(base_pairs)
                
        if position == 0:   # 'incb+sub+inv': b+a+c ===> do nothing
            return dp, short_seqname, incb_inv_pair
        
        if position == 1:  # 'inv+incb+sub': c+b+a ===> b+a+c
            inv_list, incb_list, sub_list = [list(part) for part in re.split(r'\s|\+', dp)]
            
            for bpair in base_pairs:
                strand1, pos1, strand2, pos2 = ThreeStrandReorder.get_strand_pos(bpair)
                
                # only need to change inv (c) with its connected strands
                if strand1 == 'c' and strand2 == 'b':
                    inv_list[pos1] = ')'
                    incb_list[pos2] = '('
                    incb_inv_pair = 1
                    
                if strand1 == 'c' and strand2 == 'a':
                    inv_list[pos1] = ')'
                    sub_list[pos2] = '('
        
        
        if position == 2: # 'sub+inv+incb': a+c+b ===> b+a+c
            sub_list, incb_list, inv_list = [list(part) for part in re.split(r'\s|\+', dp)]
            
            for bpair in base_pairs:
                strand1, pos1, strand2, pos2 = ThreeStrandReorder.get_strand_pos(bpair)
            
                # only need to change incb (b) with its connected strands
                if strand1 == 'a' and strand2 == 'b':
                    incb_list[pos2] = '('
                    sub_list[pos1] = ')'
                    
                if strand1 == 'c' and strand2 == 'b':
                    incb_list[pos2] = '('
                    inv_list[pos1] = ')'
        
        inv = ''.join(inv_list)
        sub = ''.join(sub_list)
        incb = ''.join(incb_list)
        # incb+sub+inv
        dp_new = incb + "+" + sub + "+" + inv
        
        return dp_new, case2[0], incb_inv_pair


    def reorderCase3(self, case3, dp, seq, short_seqname, ref_name_list, strand_list):
        """
        Reorder:
        'sub+incb inv' -> 'incb+sub inv' 
        'inv sub+incb' -> 'incb+sub inv'
        'sub+inv incb' -> 'incb+sub inv' 
        """
        
        position = case3.index(short_seqname)
        
        incb_inv_pair = 0 # certainly no incumbent-invader pair
                
        if position == 0:   # 'incb+sub inv': b+a c ===> do nothing
            return dp, short_seqname, incb_inv_pair
        
        
        base_pairs = self.get_basepairs(seq, dp, ref_name_list, strand_list)
        
        if position == 1:  # 'sub+incb inv': a+b c ===> b+a c
            sub_list, incb_list, inv_list  = [list(part) for part in re.split(r'\s|\+', dp)]
            
            for bpair in base_pairs:
                strand1, pos1, strand2, pos2 = ThreeStrandReorder.get_strand_pos(bpair)
                
                # only need to change incb (b) with its connected strand sub (a)
                if strand1 == 'a' and strand2 == 'b':
                    incb_list[pos2] = '('
                    sub_list[pos1] = ')'
        
        
        if position == 2: # 'inv sub+incb': c a+b ===> b+a c
            inv_list, sub_list, incb_list = [list(part) for part in re.split(r'\s|\+', dp)]
            
            for bpair in base_pairs:
                strand1, pos1, strand2, pos2 = ThreeStrandReorder.get_strand_pos(bpair)
                
                # only need to change incb (b) with its connected strand sub (a)
                if strand1 == 'a' and strand2 == 'b':
                    incb_list[pos2] = '('
                    sub_list[pos1] = ')'
                    
        
        if position == 3: # 'inv incb+sub': c b+a ===> b+a c
            inv_list, incb_list, sub_list = [list(part) for part in re.split(r'\s|\+', dp)]
            # just need to exchange the position of inv and incb+sub
            
        inv = ''.join(inv_list)
        sub = ''.join(sub_list)
        incb = ''.join(incb_list)
        # incb+sub inv
        dp_new = incb + "+" + sub + " " + inv
        
        return dp_new, case3[0], incb_inv_pair


    def reorderCase4(self, case4, dp, seq, short_seqname, ref_name_list, strand_list):
        """
        Reorder:
        'incb inv+sub' -> 'incb sub+inv' 
        'inv+sub incb' -> 'incb sub+inv' 
        'sub+inv incb' -> 'incb sub+inv' 
        """
        
        position = case4.index(short_seqname)
        
        incb_inv_pair = 0 # certainly no incumbent-invader pair
                
        if position == 0:   # 'incb sub+inv': b a+c ===> do nothing
            return dp, short_seqname, incb_inv_pair
        
        
        base_pairs = self.get_basepairs(seq, dp, ref_name_list, strand_list)
        
        if position == 1:  # 'incb inv+sub': b c+a ===> b a+c
            incb_list, inv_list, sub_list = [list(part) for part in re.split(r'\s|\+', dp)]
            
            for bpair in base_pairs:
                strand1, pos1, strand2, pos2 = ThreeStrandReorder.get_strand_pos(bpair)
                
                # only need to change inv (c) with its connected strand sub (a)
                if strand1 == 'c' and strand2 == 'a':
                    inv_list[pos1] = ')'
                    sub_list[pos2] = '('
                    
        
        if position == 2: # 'inv+sub incb': c+a b ===> b a+c
            inv_list, sub_list, incb_list = [list(part) for part in re.split(r'\s|\+', dp)]
            
            for bpair in base_pairs:
                strand1, pos1, strand2, pos2 = ThreeStrandReorder.get_strand_pos(bpair)
                
                # only need to change inv (c) with its connected strand sub (a)
                if strand1 == 'c' and strand2 == 'a':
                    inv_list[pos1] = ')'
                    sub_list[pos2] = '('
        
        
        if position == 3: # 'sub+inv incb': a+c b ===> b a+c
            sub_list, inv_list, incb_list = [list(part) for part in re.split(r'\s|\+', dp)]
            # just need to exchange the position of sub+inv and incb
            
        inv = ''.join(inv_list)
        sub = ''.join(sub_list)
        incb = ''.join(incb_list)
        # incb sub+inv
        dp_new = incb + " " + sub + "+" + inv
        
        return dp_new, case4[0], incb_inv_pair


    def reorderCase5(self, case5, dp, seq, short_seqname, ref_name_list, strand_list):
        """
        Reorder:
        'inv+incb sub' -> 'incb+inv sub' 
        'sub inv+incb' -> 'incb+inv sub' 
        'sub incb+inv' -> 'incb+inv sub' 
        """
        
        position = case5.index(short_seqname)
        
        incb_inv_pair = 1 # certainly having incumbent-invader pair
                
        if position == 0:   # 'incb+inv sub': b+c a ===> do nothing
            return dp, short_seqname, incb_inv_pair
        
        
        base_pairs = self.get_basepairs(seq, dp, ref_name_list, strand_list)
        
        if position == 1:  # 'inv+incb sub': c+b a ===> b+c a
            inv_list, incb_list, ub_list = [list(part) for part in re.split(r'\s|\+', dp)]
            
            for bpair in base_pairs:
                strand1, pos1, strand2, pos2 = ThreeStrandReorder.get_strand_pos(bpair)
                
                # only need to change inv (c) with its connected strand incb (b)
                if strand1 == 'c' and strand2 == 'b':
                    inv_list[pos1] = ')'
                    incb_list[pos2] = '('
                    
        
        if position == 2: # 'sub inv+incb': a c+b ===> b+c a
            sub_list, inv_list, incb_list = [list(part) for part in re.split(r'\s|\+', dp)]
            
            for bpair in base_pairs:
                strand1, pos1, strand2, pos2 = ThreeStrandReorder.get_strand_pos(bpair)
                
                # only need to change inv (c) with its connected strand incb (b)
                if strand1 == 'c' and strand2 == 'b':
                    inv_list[pos1] = ')'
                    incb_list[pos2] = '('
        
        
        if position == 3: # 'sub incb+inv': a b+c ===> b+c a
            sub_list, incb_list, inv_list = [list(part) for part in re.split(r'\s|\+', dp)]
            # just need to exchange the position of sub and incb+inv
            
        inv = ''.join(inv_list)
        sub = ''.join(sub_list)
        incb = ''.join(incb_list)
        # incb+inv sub
        dp_new = incb + "+" + inv + " " + sub
        
        return dp_new, case5[0], incb_inv_pair


    def reorderCase6(self, case6, dp, seq, short_seqname, ref_name_list, strand_list):
        """
        Reorder:
        'incb inv sub' -> 'incb sub inv' 
        'sub inv incb' -> 'incb sub inv' 
        'sub incb inv' -> 'incb sub inv' 
        'inv incb sub' -> 'incb sub inv' 
        'inv sub incb' -> 'incb sub inv' 
        """
        
        position = case6.index(short_seqname)
        
        incb_inv_pair = 0 # certainly not incumbent-invader pair
                
        if position == 0:   # 'incb sub inv': b a c ===> do nothing
            return dp, short_seqname, incb_inv_pair
            
        if position == 1:  # 'incb inv sub': b c a ===> b a c
            incb_list, inv_list, sub_list = [list(part) for part in re.split(r'\s|\+', dp)]
        
        if position == 2: # 'sub inv incb': a c b ===> b a c
            sub_list, inv_list, incb_list = [list(part) for part in re.split(r'\s|\+', dp)]
        
        if position == 3: # 'sub incb inv': a b c ===> b a c
            sub_list, incb_list, inv_list = [list(part) for part in re.split(r'\s|\+', dp)]

        if position == 4: # 'inv incb sub': c b a ===> b a c
            inv_list, incb_list, sub_list = [list(part) for part in re.split(r'\s|\+', dp)]
        
        if position == 5: # 'inv sub incb': c a b ===> b a c
            inv_list, sub_list, incb_list = [list(part) for part in re.split(r'\s|\+', dp)]
                
        inv = ''.join(inv_list)
        sub = ''.join(sub_list)
        incb = ''.join(incb_list)
        # incb sub inv
        dp_new = incb + " " + sub + " " + inv
        
        return dp_new, case6[0], incb_inv_pair
    
    def nameMap(self, sequence, strand_sub, strand_incb, strand_inv):
        def replace_strings(s):
            s = s.replace(strand_inv, "inv")
            s = s.replace(strand_incb, "incb")
            s = s.replace(strand_sub, "sub")
            return s

        vectorized_replace = np.vectorize(replace_strings)

        return vectorized_replace(np.array([sequence])).tolist()[0]
    
    def dp_reorder(self, dp, seq, ref_name_list, strand_list, strand_sub, strand_incb, strand_inv):
    
        short_seqname = self.nameMap(seq, strand_sub, strand_incb, strand_inv)
        
        case1 = ['inv+sub+incb', 'incb+inv+sub', 'sub+incb+inv']
        case2 = ['incb+sub+inv', 'inv+incb+sub', 'sub+inv+incb']
        case3 = ['incb+sub inv', 'sub+incb inv', 'inv sub+incb', 'inv incb+sub'] 
        case4 = ['incb sub+inv', 'incb inv+sub', 'inv+sub incb', 'sub+inv incb'] 
        case5 = ['incb+inv sub', 'inv+incb sub', 'sub inv+incb', 'sub incb+inv'] ##
        case6 = ['incb sub inv', 'incb inv sub', 'sub inv incb' 'sub incb inv' 'inv incb sub' 'inv sub incb']
        
        if short_seqname in case1:
            dp_new, short_seqname, incb_inv_pair = self.reorderCase1(case1, dp, seq, short_seqname, ref_name_list, strand_list)
            
        if short_seqname in case2:
            dp_new, short_seqname, incb_inv_pair = self.reorderCase2(case2, dp, seq, short_seqname, ref_name_list, strand_list)
        
        if short_seqname in case3:
            dp_new, short_seqname, incb_inv_pair = self.reorderCase3(case3, dp, seq, short_seqname, ref_name_list, strand_list)
            
        if short_seqname in case4:
            dp_new, short_seqname, incb_inv_pair = self.reorderCase4(case4, dp, seq, short_seqname, ref_name_list, strand_list)
        
        if short_seqname in case5:
            dp_new, short_seqname, incb_inv_pair = self.reorderCase5(case5, dp, seq, short_seqname, ref_name_list, strand_list)
        
        if short_seqname in case6:
            dp_new, short_seqname, incb_inv_pair = self.reorderCase6(case6, dp, seq, short_seqname, ref_name_list, strand_list)
        
        return dp_new, short_seqname, incb_inv_pair



In [16]:
reoder_3strand = ThreeStrandReorder()

num = 1560

seq = seqlabel_uniq[num]
dp = dp_og_uniq[num]
print(arr_short[num], dp)
print()

p_new, short_seqname, incb_inv_pair = reoder_3strand.dp_reorder(dp, seq, ref_name_list, strand_list, strand_sub, strand_incb, strand_inv)
print(p_new, short_seqname, incb_inv_pair)


sub+incb+inv ((....((..(((((((((((((((((+................)))))))))))))))))+......((((....)))).)))).

......((((....)))).((((.+))....))..(((((((((((((((((+................))))))))))))))))) inv+sub+incb 0


In [ ]:
case1 = ['inv+sub+incb', 'sub+incb+inv', 'incb+inv+sub',]
case2 = ['incb+sub+inv', 'inv+incb+sub', 'sub+inv+incb']
case3 = ['incb+sub inv', 'sub+incb inv', 'inv sub+incb', 'inv incb+sub'] 
case4 = ['incb sub+inv', 'incb inv+sub', 'inv+sub incb', 'sub+inv incb'] 
case5 = ['incb+inv sub', 'inv+incb sub', 'sub inv+incb', 'sub incb+inv'] ##

np.where(arr_short == case1[0])[0]

### PCA Variance

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
# Load embeddings

TNAME = '24-0214-2008'
CKPT = 'checkpoint_epoch_39'
# CKPT = 'model'

inpath5 = f'../data/post_data/{datafile}/model_config/{TNAME}/embed_{CKPT}_Machinek-PRF.npz'
loaded_data5 = np.load(inpath5, allow_pickle=True)
embeddings = loaded_data5['embeddings']
embeddings.max(), embeddings.min(), embeddings.mean(), embeddings.std()

In [ ]:
cm = PCA(n_components=25)
cm.fit(embeddings)

PC_values = np.arange(cm.n_components_) + 1
plt.plot(PC_values, np.cumsum(cm.explained_variance_ratio_), 'ro-', linewidth=2)
plt.title('Scree Plot: PCA')
plt.xlabel('Number of principal components')
plt.ylabel('Cumulative explained variance');
# plt.xticks(np.arange(0, data_embed.shape[-1]+1, 1))

plt.show()

print(np.cumsum(cm.explained_variance_ratio_))

### Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


datafile = 'Machinek-PRF-trunc'

inpath = f'../data/post_data/{datafile}/preprocess_Machinek-PRF.npz'
loaded_data = np.load(inpath, allow_pickle=True)
trans_time = loaded_data["trans_time"]

inpath4 = f'../data/post_data/{datafile}/time_Machinek-PRF.npz'
loaded_data4 = np.load(inpath4, allow_pickle=True)
trj_id = loaded_data4['trj_id']


In [ ]:
rxnTimes = trans_time[trj_id]
ids = np.arange(rxnTimes.shape[0])

sortTimes = np.sort(rxnTimes)[::-1]
sortIDs = ids[np.argsort(rxnTimes)[::-1]]


In [ ]:
fig = px.scatter(x=ids, y=sortTimes, hover_data=[sortIDs], log_y=True)
fig.show()

In [ ]:
np.mean(sortTimes), np.max(sortTimes), np.min(sortTimes)